In [ ]:
import rioxarray as rxr
import gval
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib import cm, colors

In [ ]:

benchmark = rxr.open_rasterio(
    'C:/.../.tif',
)
benchmark = benchmark.dropna(dim="y", how="all").dropna(dim="x", how="all")
candidate = rxr.open_rasterio(
    'C:/.../.tif',
)

In [ ]:
candidate = xr.where(candidate > 0, 1, 0).rio.write_nodata(0, inplace=True)
benchmark = xr.where(benchmark > 0, 1, 0).rio.write_nodata(0, inplace=True)

In [ ]:
candidate, benchmark = candidate.gval.homogenize(
    benchmark_map=benchmark,
)
candidate = candidate.sel( x = benchmark['x'], y = benchmark['y'],)
candidate.plot()

In [ ]:
# riv =  gpd.read_file('examples/verdigris_10m_v8/segs/fldsensing_segments.shp' ,)
# riv = riv.to_crs(candidate.rio.crs)
# riv.geometry = riv.geometry.buffer(90)

In [ ]:
# mask = candidate.rio.clip(riv.geometry.apply(mapping), riv.crs, drop=True)
# mask = xr.where(~np.isnan(mask), 1, np.nan)
# candidate = xr.where(mask == 1, np.nan, candidate)
# benchmark = xr.where(mask == 1, np.nan, benchmark)

In [ ]:
agreement_map, crosstab_table, metric_table = candidate.gval.categorical_compare(
    benchmark, positive_categories=[1], average = 'micro'
)
metric_table.transpose()

In [ ]:
agreement_map.gval.cat_plot(figsize = (5,8), legend_labels = ['TN','FN','FP','TP'], 
                            basemap = None, colormap = 'jet', title = 'RS-FLDPLN FIM')

# Plot

In [ ]:
def replace_values(data_array):
    # Replace values in the data array with corresponding indices
    replaced_data = xr.where(data_array == 0, 0,
                             xr.where(data_array == 4, 1,
                                      xr.where(data_array == 6, 2,
                                               xr.where(data_array == 8, 3, data_array))))
    return replaced_data

agreement_map1 = replace_values(agreement_map)

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm

levels = [0, 1, 2, 3, 4]
labels = ["TN", "FN", "FP", "TP"]
cmap = ListedColormap(['grey', 'red', 'orange', 'green'])
norm = BoundaryNorm(levels, cmap.N)

fig, ax = plt.subplots(figsize=(20, 10))
plot = agreement_map1.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False, add_labels=False)

ax.axis('off')
ax.set_aspect('equal')
ax.set_xlim(float(agreement_map1.x.min()), float(agreement_map1.x.max()))
ax.set_ylim(float(agreement_map1.y.min()), float(agreement_map1.y.max()))
fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

ax.text(0.03, 0.03,
        "CSI = {:.2f}\nPOD = {:.2f}\nFAR = {:.2f}\nF-1 = {:.2f}".format(
            metric_table['critical_success_index'][0],
            metric_table['true_positive_rate'][0],
            metric_table['false_discovery_rate'][0],
            metric_table['f_score'][0]),
        transform=ax.transAxes, fontsize=15, va='bottom', ha='left', color ='white')

ax.text(0.01, 0.99, "(b.)", transform=ax.transAxes, fontsize=16,
        verticalalignment='top', horizontalalignment='left', color='white')

plt.show()

In [ ]:
tif_path = r"C:/.../.tif"
with rasterio.open(tif_path) as src:
    img = src.read(1)
    nodata = src.nodata
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

if nodata is not None:
    img = np.where(img == nodata, np.nan, img)
img = np.where(img == 0, np.nan, img)

vmin = 0
vmax = np.ceil(np.nanmax(img) * 100) / 100  

cmap = cm.get_cmap('Blues').copy()
cmap.set_bad(color='grey') 

fig, ax = plt.subplots(figsize=(20, 10))
im = ax.imshow(img, extent=extent, cmap=cmap, origin='upper',
               vmin=vmin, vmax=vmax)

ax.set_xlim(float(agreement_map1.x.min()), float(agreement_map1.x.max()))
ax.set_ylim(float(agreement_map1.y.min()), float(agreement_map1.y.max()))

ax.axis('off')
ax.set_aspect('equal')
fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

cax = inset_axes(ax, width="3%", height="30%", loc='lower left', borderpad=2)
cbar = fig.colorbar(im, cax=cax)
cbar.set_label('Depth (feet)', fontsize=12, color='white') 
cbar.ax.tick_params(labelsize=10, colors='white')   

ax.text(0.01, 0.99, "(a.)", transform=ax.transAxes, fontsize=16,
        verticalalignment='top', horizontalalignment='left', color='white')

plt.show()

In [ ]:
agreement_map = agreement_map.rio.write_crs("EPSG:4326")

In [ ]:
regions = [-95.7348872270929, -95.50682323623167, 36.991503881271214, 37.24254955524226]

In [ ]:
agreement_map.gval.cat_plot(figsize = (5,8), legend_labels = ['TN','FN','FP','TP'], 
                            basemap = None, colormap = 'jet', title = 'RS-FLDPLN FIM')

# RMSE

In [ ]:
import rioxarray as rxr
import gval
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
benchmark = rxr.open_rasterio(
    'C:/.../.tif',
)
benchmark = benchmark.dropna(dim="y", how="all").dropna(dim="x", how="all")

candidate = rxr.open_rasterio(
     'C:/.../.tif',
)

candidate.rio.write_nodata(-9999, inplace=True)
benchmark.rio.write_nodata(-9999, inplace=True)

candidate, benchmark = candidate.gval.homogenize(
    benchmark_map=benchmark,
)
candidate = candidate.sel( x = benchmark['x'], y = benchmark['y'], method='nearest')
candidate.plot()

In [ ]:
benchmark.plot()

In [ ]:
sim = candidate.values[0]
bench = benchmark.values[0]
res = agreement_map.values[0]

res

In [ ]:
sim[res != 8] = np.nan
bench[res != 8] = np.nan

In [ ]:
benchmark_flat = bench.flatten()
benchmark_flat = benchmark_flat[~np.isnan(benchmark_flat)]

plt.hist(benchmark_flat, bins=20, edgecolor='black')
plt.title("HEC-RAS Depth")
plt.xlabel("Depth")
plt.ylabel("Frequency")
plt.show()

In [ ]:

candidate_flat = sim.flatten()
candidate_flat = candidate_flat[~np.isnan(candidate_flat)]

plt.hist(candidate_flat, bins=20, edgecolor='black')
plt.title("FLDSensing Depth")
plt.xlabel("Depth")
plt.ylabel("Frequency")
plt.show()

In [ ]:
diff = sim - bench
diff

In [ ]:
np.nanmax(diff) , np.nanmin(diff)

In [ ]:
def pbias(observed, simulated):

    observed_flat = observed.flatten()
    simulated_flat = simulated.flatten()
    
    mask = ~np.isnan(observed_flat) & ~np.isnan(simulated_flat)
    
    observed_valid = observed_flat[mask]
    simulated_valid = simulated_flat[mask]
    
    bias = np.sum(simulated_valid-observed_valid)
    total = np.sum(observed_valid)

    pbias_value = (bias / total) * 100
    return pbias_value

def rmse(simulated, observed):
    simulated = simulated.flatten()
    observed = observed.flatten()
    
    mask = ~np.isnan(simulated) & ~np.isnan(observed)
    sim_valid = simulated[mask]
    obs_valid = observed[mask]
    
    return np.sqrt(np.mean((sim_valid - obs_valid) ** 2))

In [ ]:
rmse(sim, bench).round(2)

In [ ]:
pbias(bench,sim).round(2)

In [ ]:
np.nanmax(diff) , np.nanmin(diff)

In [ ]:
cmap = matplotlib.cm.seismic.copy()
cmap.set_bad('grey', 1.)

diff = sim - bench
vmax = np.nanmax(np.abs(diff))

fig, ax = plt.subplots(figsize=(20, 10))

im = ax.imshow(diff, cmap=cmap, vmin=-vmax, vmax=vmax)

cax = inset_axes(ax, width="3%", height="30%", loc='lower left', borderpad=2)
cbar = fig.colorbar(im, cax=cax)
cbar.set_label("Flood depth difference (Feet)", fontsize=12, color='white')
cbar.ax.tick_params(labelsize=10, colors='white')

ax.text(0.99, 0.90, f"RMSE = {rmse(sim, bench):.2f}",
        transform=ax.transAxes, fontsize=15, va='top', ha='right', color='white')
ax.text(0.99, 0.96, f"PBIAS = {pbias(bench, sim):.2f}%",
        transform=ax.transAxes, fontsize=15, va='top', ha='right', color='white')

ax.text(0.01, 0.99, "(c.)", transform=ax.transAxes, fontsize=16,
        verticalalignment='top', horizontalalignment='left', color='white')

ax.axis('off')
ax.set_aspect('equal')
fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

plt.show()